# Extract vina results

This notebook then creates the new csv `data_all.csv` which contains all the results from this project neccessary to create the metrics figure.

In [19]:
import pandas as pd  
import glob

In [38]:
# SILVR + EDM, fragments 0072 + 0107
# experiment_names = ['exp_9','exp_10','exp_11','exp_12','exp_13','exp_14','exp_15']

# BRIDGE + EDM, fragments 0072 + 0107
# experiment_names = ['exp_16','exp_17','exp_18','exp_19','exp_20','exp_21','exp_22']
# BRIDGE + EDM, fragments 0072 + 0107 (250 SAMPLES)
# experiment_names = ['exp_23','exp_24','exp_25','exp_26','exp_27','exp_28','exp_29']

# BRIDGE + EDM, fragments 0072 + 0107 (Explicit Hydrogens)
experiment_names = ['exp_30','exp_31','exp_32','exp_33','exp_34','exp_35','exp_36']
#TODO: Fix inconsistent naming of variables
exp_names = experiment_names
vina_exp_list = [f"docking_data/{x}/" for x in exp_names]#vina without water
# vina_exp_list = [f"docking_data/{x}/" for x in exp_names] # TODO: vina with water
experiment_path = "experiments/"

In [39]:
# vina scores are already written to a csv file so there's no need to extract even further.
vina_df_list = [f"{x}_mpro_vina_docking_data.csv" for x in exp_names]
vina_results_df = pd.DataFrame()
# read in all the df from vina_df_list
for path in vina_df_list:
    vina_df = pd.read_csv(path)
    # get rid of the first row if the first column is not 'ligand_name'
    if vina_df.columns[0] != 'ligand_name':
        vina_df = vina_df.iloc[1:]
        #drop the error_message column
        vina_df = vina_df.drop(columns=['error_message'])
        #rename the columns
        vina_df = vina_df.rename(columns={'computed_dG':'error_message','protein_name':'computed_dG','ligand_name':'protein_name','Unnamed: 0':'ligand_name'})
    
    vina_df_subset = vina_df[['ligand_name', 'computed_dG']]
    df = pd.DataFrame(vina_df_subset)
    df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
    vina_results_df = pd.concat([vina_results_df, df])
vina_results_df = vina_results_df.rename(columns={'ligand_name': 'mol_id', 'computed_dG': 'vina_score'})
vina_results_df

/tmp/ipykernel_3530415/3674054974.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530415/3674054974.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530415/3674054974.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp/ipykernel_3530415/3674054974.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ligand_name'] = df['ligand_name'].str.replace('.sdf', '').str.replace('.pdbqt', '').str.replace('.pdb', '')
/tmp

,mol_id,vina_score
0,mol_2024_07_10_1800131_000,-5.319
1,mol_2024_07_10_1754523_000,NaN
2,mol_2024_07_10_1749093_000,NaN
3,mol_2024_07_10_1757000_000,-5.423
4,mol_2024_07_10_1745590_000,NaN
...,...,...
99,mol_2024_07_10_2047213_000,NaN
100,mol_2024_07_10_2044152_000,NaN
101,mol_2024_07_10_2040111_000,-34.27
102,mol_2024_07_10_2039161_000,NaN


In [40]:
old_df = pd.read_csv(experiment_path+f"{experiment_names[0]}_to_{experiment_names[-1]}_data_all.csv")
# merge the two dataframes
df = pd.merge(old_df,vina_results_df, on="mol_id",how='left')
#!!! This over writes the results from earlier!!!! 
df.to_csv(experiment_path+f"{experiment_names[0]}_to_{experiment_names[-1]}_data_all.csv", index=False)
df

,mol_id,total_atoms,stable_ratio,exp_id,dummy_atoms,samples,silvr,comment,crude_rmsd,is_fragmented,qed,sa,smiles,auto3d_rmsd,vina_score
0,mol_2024_07_10_1802383_000,24,0.791667,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.176292,False,0.748112,6.850152,[H]c1sc2c(c1[H])=NN([H])c1nc(C([H])([H])[H])c(...,0.659048,-6.502
1,mol_2024_07_10_1746031_000,24,0.750000,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.468629,False,0.115395,4.058768,[H]N(N=[N+]=[N-])NC([H])(NNC([H])(OC(=O)C([H])...,1.751093,-5.741
2,mol_2024_07_10_1751470_000,24,0.708333,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.623005,True,0.648801,NaN,[H]C([H])([H])N(C([H])([H])C([H])([H])[H])S(O)...,NaN,NaN
3,mol_2024_07_10_1802540_000,24,0.916667,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.186990,False,0.467546,7.333780,[H]Oc1c([H])c(N([H])[H])c([H])c(=O)c(N([H])C([...,0.874503,-5.062
4,mol_2024_07_10_1806243_000,24,0.708333,exp_30,0,100,0.00,(BRIDGE + EDM) Effect of SILVR rate on samplin...,5.255156,False,0.145665,7.434265,[H]N(C(=O)N1C([H])(C([H])([H])NN([H])C([H])([H...,1.124797,-4.848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,mol_2024_07_10_2047480_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.172006,True,0.212535,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF.FF.FF.F...,NaN,NaN
724,mol_2024_07_10_2053181_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.139319,True,0.212511,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF...,NaN,NaN
725,mol_2024_07_10_2031292_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.146521,True,0.212547,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.FF.FF.FF.FF.FF.FF.FF...,NaN,NaN
726,mol_2024_07_10_2039161_000,47,0.000000,exp_36,0,100,0.03,(BRIDGE + EDM) Effect of SILVR rate on samplin...,4.201126,True,0.212489,NaN,F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F.F....,NaN,NaN
